# Agentic AI Tutorial: Building an Intelligent Agent

In [1]:
!pip install transformers accelerate duckduckgo-search requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

## Part 1: Setting Up the Language Model

We'll use the FLAN-T5 model from Google as our base language model. This model is:
- Instruction-tuned: Better at following specific instructions
- Relatively small: Suitable for educational purposes
- Versatile: Can handle various types of tasks

The model will serve as the "brain" of our agent, helping it decide which tools to use for different tasks.


In [2]:
from transformers import pipeline

# Load a small public model
llm = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=100)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


## Part 2: Building Our First Tool - Calculator

Let's start by implementing a simple calculator tool. This will help us understand the basic structure of tool-using agents.

Key concepts:
1. Tool Definition: A function that performs a specific task
2. Tool Registry: A dictionary that maps tool names to their implementations
3. Prompt Engineering: Creating clear instructions for the model
4. Agent Loop: The main decision-making cycle

### Understanding the Code:
- The `calculator` function uses Python's `eval` to compute mathematical expressions
- The `tools` dictionary serves as our tool registry
- The `prompt_template` provides instructions to the language model
- The `agent_loop` manages the interaction between user, model, and tools


In [ ]:
# Tool 1: Calculator
def calculator(query):
    try:
        return str(eval(query))
    except Exception as e:
        return f"Error: {e}"

tools = {
    "calculator": calculator,
}

prompt_template = """You are an intelligent reasoning agent. You must always answer questions by planning and then using one of the available tools.

Available tools:
- calculator: use this for computing math expressions like 5*(3+2), 12/4, etc.

Instructions:
- Specify which tool to use for this task.

Use the format:
Question: <question>
Thought: <tool name>

Now answer:

Question: {question}
Thought:"""

def agent_loop():
    while True:
        question = input("\nAsk me something: ")
        if question.lower() in ["exit", "quit"]:
            break

        prompt = prompt_template.format(question=question)
        thought = llm(prompt)[0]['generated_text'].split("Thought:")[-1].strip()
        print("Thought:", thought)
        
        if "calculator" in thought.lower():
            expr = ''.join(c for c in question if c.isdigit() or c in "+-*/.()")
            result = tools["calculator"](expr)
            print("Action: calculator")
            print("Input:", expr)
            print("Output:", result)
        else:
            print("Final Answer:", thought)

In [4]:
agent_loop()


Ask me something: What is 2*2?
Thought: Calculator: use this for computing math expressions like 5*(3+2), 12/4, etc.
Action: calculator
Input: 2*2
Output: 4

Ask me something: exit


### Exercise 1: Understanding the Agent Loop

Try the agent with different mathematical expressions. Consider the following questions:

1. What happens if you input an invalid mathematical expression?
2. How does the agent decide to use the calculator tool?
3. What are the limitations of this simple calculator implementation?

Try these examples:
- 2*2
- 3+5*2
- sqrt(16) (Will this work? Why/why not?)


## Part 3: Adding Web Search Capabilities

Now we'll expand our agent's capabilities by adding a web search tool. This demonstrates how agents can interact with external services.

Key additions:
1. Web Search Tool: Using DuckDuckGo's search API
2. Multiple Tool Handling: How the agent chooses between tools


In [ ]:
from duckduckgo_search import DDGS

# Tool 2: Real web search via DuckDuckGo
def web_search(query):
    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=1)
            for r in results:
                return f"{r['title']}: {r['body']}"
        return "No results found."
    except Exception as e:
        return f"Search error: {e}"

tools = {
    "calculator": calculator,
    "web_search": web_search,

}

prompt_template = """You are an intelligent reasoning agent. You must always answer questions by planning and then using one of the available tools.

Available tools:
- calculator: use this for computing math expressions like 5*(3+2), 12/4, etc.
- web_search: use this to look up facts, names, places, or current events

Instructions:
- Specify which tool to use for this task.

Use the format:
Question: <question>
Thought: <tool name>

Now answer:

Question: {question}
Thought:"""

def agent_loop():
    while True:
        question = input("\nAsk me something: ")
        if question.lower() in ["exit", "quit"]:
            break

        prompt = prompt_template.format(question=question)
        output = llm(prompt)[0]['generated_text']
        thought = output.split("Thought:")[-1].strip()
        print("Thought:", thought)

        if "calculator" in thought.lower():
            expr = ''.join(c for c in question if c.isdigit() or c in "+-*/.()")
            result = tools["calculator"](expr)
            print("Action: calculator")
            print("Input:", expr)
            print("Output:", result)

        elif "web_search" in thought.lower():
            result = tools["web_search"](question)
            print("Action: web_search")
            print("Input:", question)
            print("Output:", result)

        else:
            print("Final Answer:", thought)


In [9]:
agent_loop()


Ask me something: Who is the prime minister of Canada


/tmp/ipython-input-5-2404756251.py:13: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Thought: web_search: use this to look up facts, names, places, or current events
Action: web_search
Input: Who is the prime minister of Canada
Output: Prime Video: Enjoy exclusive Amazon Originals as well as popular movies and TV shows. Watch anytime, anywhere. Start your free trial.

Ask me something: exit


### Exercise 2: Testing Web Search Integration

Experiment with the enhanced agent by asking various questions:

1. Try factual queries (e.g., "Who is the prime minister of Canada?")
2. Try mathematical queries (How does the agent choose between calculator and web search?)
3. Try ambiguous queries (What happens?)

Questions to consider:
1. How reliable is the web search tool?
2. What are the limitations of using a single search result?
3. How could we improve the tool selection process?

## Part 4: Adding Weather Information

We'll now add a weather tool that demonstrates how to:
1. Make HTTP requests to external APIs
2. Handle real-time data
3. Process and format responses
4. Handle multiple tools with similar contexts

In [ ]:
import requests

# Tool 3: retreive weather
def get_weather(city):
    try:
        # Format: wttr.in/<city>?format=3 gives a 1-line summary
        url = f"https://wttr.in/{city}?format=3"
        response = requests.get(url)
        if response.status_code == 200:
            return response.text.strip()
        return f"Error: Could not get weather for {city}"
    except Exception as e:
        return f"Weather lookup failed: {e}"

tools = {
    "calculator": calculator,
    "web_search": web_search,
    "get_weather": get_weather,
}

prompt_template = """You are an intelligent reasoning agent. You must always answer questions by planning and then using one of the tools.

Available tools:
- calculator: use this for computing math expressions like 5*(3+2), 12/4, etc.
- web_search: use this to look up facts, names, places, or current events
- get_weather: get current weather for a city (e.g., Toronto)

Instructions:
- Specify which tool to use for this task.

Use the format:
Question: <question>
Thought: <tool name>

Now answer:

Question: {question}
Thought:"""


def agent_loop():
    while True:
        question = input("\nAsk me something: ")
        if question.lower() in ["exit", "quit"]:
            break

        prompt = prompt_template.format(question=question)
        output = llm(prompt)[0]['generated_text']

        thought = output.split("Thought:")[-1].strip()
        print("Thought:", thought)

        if "calculator" in thought.lower():
            expr = ''.join(c for c in question if c.isdigit() or c in "+-*/.()")
            result = tools["calculator"](expr)
            print("Action: calculator")
            print("Input:", expr)
            print("Output:", result)

        elif "web_search" in thought.lower():
            result = tools["web_search"](question)
            print("Action: web_search")
            print("Input:", question)
            print("Output:", result)

        elif "get_weather" in thought.lower():
            # Extract city name (could improve)
            city = question.split()[-1]
            result = tools["get_weather"](city)
            print("Action: get_weather")
            print("Input:", city)
            print("Output:", result)

        else:
            print("Final Answer:", thought)

In [11]:
agent_loop()


Ask me something: What is the weather in Toronto?
Thought: get_weather: get current weather for a city (e.g., Toronto)
Action: get_weather
Input: Toronto?
Output: Weather report: Toronto

      \   /     Sunny
       .-.      +23(24) °C     
    ― (   ) ―   ↙ 4 km/h       
       `-’      24 km          
      /   \     0.0 mm         
                                                       ┌─────────────┐                                                       
┌──────────────────────────────┬───────────────────────┤  Mon 21 Jul ├───────────────────────┬──────────────────────────────┐
│            Morning           │             Noon      └──────┬──────┘     Evening           │             Night            │
├──────────────────────────────┼──────────────────────────────┼──────────────────────────────┼──────────────────────────────┤
│     \   /     Sunny          │     \   /     Sunny          │     \   /     Sunny          │     \   /     Clear          │
│      .-.      18 °C          